In [1]:
import randomname
import json
import random
import time
import string
from essential_generators import DocumentGenerator

In [2]:
USERS = 40
ACTIVE_CHALLENGES = 12
FINISHED_CHALLENGES = 15
VOTE_PERC = 0.7
OUT_PATH = "../data/presets/"

In [3]:
%%time

# generate users
#     {
#         "name": "Test 1",
#         "email": "test1@foo.bar",
#         "password": "test"
#     }

users = []

for i in range(USERS):
    user = {
        "name": randomname.get_name(),
        "email": f'test{i}@test.com',
        "password": "test",
    }
    users.append(user)

CPU times: user 65 ms, sys: 3.47 ms, total: 68.5 ms
Wall time: 77.4 ms


In [4]:
def str_time_prop(start, end, time_format, prop):
    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(time_format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [5]:
%%time

# generate challenges
#   {
#     "type": "bool",
#     "content": "Победит ли Украина Россию?",
#     "description": "Одержит ли Украина победу в противостоянии с Россией?",
#     "startTime": "2022-02-24",
#     "endTime": "2022-04-01",
#     "timeLayout": "2006-01-02"
#   }

active_challenges = []
finished_challenges = []

gen = DocumentGenerator()

for i in range(ACTIVE_CHALLENGES):
    ch = {
        "type": "bool",
        "content": gen.sentence(),
        "description": "Думаете ли вы что это верно? Test active",
        "startTime": random_date("2022-02-01", "2022-03-25", random.random()),
        "endTime": random_date("2023-01-01", "2023-12-30", random.random()),
        "timeLayout": "2006-01-02"
    }
    active_challenges.append(ch)
    
for i in range(FINISHED_CHALLENGES):
    ch = {
        "type": "bool",
        "content": gen.sentence(),
        "description": "Думаете ли вы что это верно? Test finished",
        "startTime": random_date("2021-02-01", "2021-03-25", random.random()),
        "endTime": random_date("2022-01-01", "2022-03-12", random.random()),
        "timeLayout": "2006-01-02",
        "outcome": random.choice([True, False])
    }
    finished_challenges.append(ch)
    
challenges = active_challenges + finished_challenges

CPU times: user 1.39 s, sys: 356 ms, total: 1.75 s
Wall time: 1.79 s


In [6]:
%%time

# generate predictions
#     {
#         "userEmail": "test1@foo.bar",
#         "challengeContent": "Россия нападёт на Украину",
#         "prognosis": true
#     }

predictions = []

for user in users: 
    for ch in challenges:
        # some users may skip some challenges
        if random.random() < VOTE_PERC:
            pred = {
                "userEmail": user["email"],
                "challengeContent": ch["content"],
                "prognosis": random.choice([True, False]),
            }
            predictions.append(pred)
            
print("generated predictions:", len(predictions))

generated predictions: 1109
CPU times: user 10.2 ms, sys: 0 ns, total: 10.2 ms
Wall time: 9.97 ms


In [7]:
%%time

def save_file(file, data):
    with open(OUT_PATH + file, 'w', encoding='utf8') as out:
        json.dump(data, out, indent=2, ensure_ascii=False)
    
save_file('users_auto.json', users)
save_file('challenges_auto.json', challenges)
save_file('predictions_auto.json', predictions)

CPU times: user 6.56 ms, sys: 0 ns, total: 6.56 ms
Wall time: 11.5 ms
